In [1]:
import cv2
import tonic
import os
import numpy as np
import matplotlib.pyplot as plt
import tonic.transforms as transforms
import tonic.slicers as slicers

from tqdm import tqdm
from EvEye.utils.tonic.slicers.SliceWithTimestampAndCount import (
    slice_events_by_timestamp_and_count,
)
from EvEye.utils.tonic.slicers.SliceEventsAtIndices import slice_events_at_timepoints
from EvEye.utils.cache.MemmapCacheStructedEvents import *
from EvEye.utils.visualization.visualization import visualize, save_image
from EvEye.utils.processor.TxtProcessor import TxtProcessor
from EvEye.utils.tonic.functional.ToFrameStack import to_frame_stack_numpy
from EvEye.utils.tonic.functional.CutMaxCount import cut_max_count
from EvEye.utils.tonic.functional.PlotDistribution import plot_histogram, plot_KDE

/home/junyuan/anaconda3/envs/EvEye/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "/mnt/data2T/junyuan/eye-tracking/datasets/DavisEyeCenterDataset/test/data/user43_left_session_1_0_1_events.txt"
output_path = "/mnt/data2T/junyuan/eye-tracking/outputs/CountsRange"
txt_processor = TxtProcessor(data_path)
events = txt_processor.load_events_from_txt()
events['t'] -= events['t'][0]
segment_num = events['t'][-1] // 40000
sensor_size = (346, 260, 2)
time_window = 40000
start = 0
interval_length = 200
interval_num = 100

In [3]:
def get_range(start, interval_length, interval_num):
    counts = []
    for index in range(interval_num):
        start_point = start + index * interval_length
        end_point = start_point + interval_length
        counts.append((start_point, end_point))
    return counts

In [4]:
counts_range = get_range(start, interval_length, interval_num)
for index in range(segment_num):
    start_time = index * time_window
    end_time = (index + 1) * time_window
    event_segement = slice_events_at_timepoints(events, start_time, end_time)
    event_counts = event_segement['t'].shape[0]
    for count_range in counts_range:
        if count_range[0] <= event_counts <= count_range[1]:
            event_frame = to_frame_stack_numpy(
                event_segement, sensor_size, 1, "causal_linear", start_time, end_time
            )
            event_frame_vis = visualize(event_frame)
            # print(f"Segment {index} with {event_counts} events")
            folder_path = f"{output_path}/{count_range[0]:05d}-{count_range[1]:05d}"
            os.makedirs(folder_path, exist_ok=True)
            save_image(
                event_frame_vis,
                f"{folder_path}/segment_{index}_with_{event_counts}_events.png",
                BGR2RGB=True,
            )

In [ ]:
cache_data_path = "/mnt/data2T/junyuan/eye-tracking/datasets/MemmapDavisEyeCenterDataset/train/cached_data"
cache_label_path = "/mnt/data2T/junyuan/eye-tracking/datasets/MemmapDavisEyeCenterDataset/train/cached_label"
events_list = load_cached_structed_events(cache_data_path)
labels_list = load_cached_structed_labels(cache_label_path)

In [ ]:
event = events_list[0]
label = labels_list[0]
sensor_size = (346, 260, 2)

In [ ]:
label

In [ ]:
frame_transform = transforms.ToFrame(
    sensor_size=sensor_size, time_window=40000, include_incomplete=True
)
frames = frame_transform(event)
frames_vis = visualize(frames)
frames.shape, frames_vis.shape

In [ ]:
"""Show the frames with slice_events_by_timestamp_and_count"""

count = 5000
for index in range(len(label)):
    event_segement = slice_events_by_timestamp_and_count(
        event, label["t"][index], count
    )
    transform = transforms.ToFrame(
        sensor_size=sensor_size, n_event_bins=1, include_incomplete=True
    )
    frame = transform(event_segement)
    frame_vis = visualize(frame)
    center_x, center_y = label["x"][index], label["y"][index]
    cv2.circle(frame_vis, (center_x, center_y), 3, (0, 255, 0), -1)
    plt.imshow(frame_vis)
    plt.axis("off")
    plt.show()

In [ ]:
"""Show the frames"""

for index in range(100, 200):
    # frame_vis = np.concatenate([frames_vis[index], frames_vis[index]], axis=1)
    # frame_vis = cv2.cvtColor(frame_vis, cv2.COLOR_BGR2RGB)
    center_x, center_y = label["x"][index], label["y"][index]
    cv2.circle(frames_vis[index], (center_x, center_y), 3, (0, 255, 0), -1)
    plt.imshow(frames_vis[index])
    plt.axis("off")
    plt.show()

In [ ]:
"""Save the frames"""

# for index in tqdm(total=range(frames_vis.shape[0]), desc="Saving images"):
#     center_x, center_y = label['x'][index], label['y'][index]
#     cv2.circle(frames_vis[index], (center_x, center_y), 3, (0, 255, 0), -1)
#     save_image(frames_vis[index], f"/mnt/data2T/junyuan/eye-tracking/some/{index}.png")